# Курс Рекомендательные системы

# Практическое задание урока 3. Коллаборативная фильтрация
1. Попытаться ответить на вопросы/выдвинуть гипотезы

2. Доделать прошлые домашния задания

3. Прочитать статьи BM25/MatrixFactorization

4. Поэкспериментировать с ALS (grid-search)

## Задание 1. 

**Попытаться ответить на вопросы/выдвинуть гипотезы**

Пояснение: 

Нужно попробовать ответить на вопросы и выдвинуть гипотезы как это можно решить:

* **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)

* **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?)

* **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

# Production

Начиная с этого вебинара, мы будем строить *базовое решение* для системы рекомендаций топ-N товаров. В финальном проекте вам нужно будет его сущесвтенно улучшить.  
  
**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель)
- По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)
- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?
- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?
- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)
- Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?
- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки
- Cколько мы хотим зарабатывать с одного привлеченного юзера?
---
- А точно нужно сортировать по вероятности?
- Какую метрику использовать?
- Сколько раз в неделю отпрпавляем рассылку?
- В какое время отправляем рассылку?
- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?
- И многое другое:)

**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влезает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)
- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) 
- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

Вопросов стало еще больше. Поэтому сначала делаем **MVP** (Minimum viable product) на e-mail. Показываем его менеджеру, измеряем метрики на юзерах. По фидбеку и метрикам делаем улучшения MVP и раскатываем его на push-уведомления и чеки

*Data Science проект* - итеративный процесс!

#### Решение
**3 товара с акцией**  *(Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)*

Можно использовать стретегию объединения моделей -  считать средневзвешенный прогноз по нескольким оценкам, оценке количества покупок и оценке цены товара. 

**1 новый товар** *(юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?)*

Новый товар - это некая неизвестность. Не хотелось бы, чтобы предложение повлияло на лояльность клиента негативно, зато позитивное влияние на лояльность - это отвечает вышеуказанной задаче повышения роста удержания минимум на 3%. Можно использовать логику "часто мы не валидируем поведение, а валидируем свои ценности". Новый товар, противоречащий ценностям клиента, следует однозначно исключить из рассылки (не предлагать свиной окорок юзерам, потребляющим продукты категорий веган, кошер или халяль; не предлагать продукт в тетрапаке юзерам, приобретающим товары в экологичной упаковке и т.п.).  Можно рандомно предлагать новый товар из подборки товаров, совпадающих с ценностями юзера. Даже если такой товар не будет куплен, то будет оценен юзером позитивно ("этот магазин разделяет мои ценности"). Как следствие, возрастет удержание.
Вариант решения - считать отсутствие покупок в категории товаров, к которой относится новый товар, отрицательной оценкой юзера.

**1 товар для роста среднего чека** *(товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)*

В соответствии с вышеизложенными договоренностями выручку будем повышать за счет роста удержания клиентов минимум на 3% и среднего чека минимум на 3%. Не хотелось бы, чтобы рост среднего чека привел к оттоку клиентов. Возьмем группы клиентов со схожими обычными покупками. Внутри группы разделим юзеров на две подгруппы по размеру среднего чека, например, по медиане. Юзерам с размером среднего чека ниже медианного будем предлагать подобный товар, который обычно покупают юзеры с размером среднего чека выше медианного.

## Задание 3
**Прочитать статьи BM25/MatrixFactorization**

Выполнено. Ссылки:

BM25 https://en.wikipedia.org/wiki/Okapi_BM25#:~:text=BM25%20is%20a%20bag%2Dof,slightly%20different%20components%20and%20parameters.

Matrix factorization (ALS, SVD) https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/

## Задание 4
**Поэкспериментировать с ALS (grid-search)**

Исчерпывающую информацию с теорией, кодом и примерами можно найти в [статье](https://www.ethanrosenthal.com/2016/10/19/implicit-mf-part-1/)

***Решение***

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

In [2]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [3]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [6]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

### 1. Подбор матрицы весов

**1.1 Базовая модель**

In [8]:
csr_mat = csr_matrix(user_item_matrix).T.tocsr()  # Применяется к item-user матрице ! 

In [9]:
%%time

model = AlternatingLeastSquares(factors=100, #k
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=2)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 8.7 s


In [10]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

Wall time: 1min 12s


0.16268364348677536

**1.2 TF-IDF взвешивание**

In [11]:
user_item_matrix_tfidf = tfidf_weight(user_item_matrix.T).T  # Применяется к item-user матрице !

In [12]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=2)

model.fit(csr_matrix(user_item_matrix_tfidf).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_tfidf'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 1min 19s


0.16238981390793122

**1.3 BM25 взвешивание**

In [13]:
user_item_matrix_bm25 = bm25_weight(user_item_matrix.T).T

In [14]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix_bm25).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 1min 3s


0.1895200783545516

**Вывод результатов**

In [15]:
models = result.columns[2:]
score = []
for model_ in models:
    score.append(result.apply(lambda x: precision_at_k(x[model_], x['actual'], k=5), axis=1).mean())

In [16]:
evaluation = pd.DataFrame({'model': models, 'precision': score})
evaluation.sort_values('precision', ascending=False)

,model,precision
2,als_bm25,0.189520
0,als,0.162684
1,als_tfidf,0.162390


### 2. Оптимизация гиперпараметров

In [17]:
import itertools
from time import time

In [18]:
result_test = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns=['user_id', 'actual']

result_train = data_train.groupby('user_id')['item_id'].unique().reset_index()
result_train.columns=['user_id', 'actual']

In [19]:
def create_model(data, factors, regularization, iterations):
    model = AlternatingLeastSquares(factors=factors, 
                                    regularization=regularization,
                                    iterations=iterations, 
                                    calculate_training_loss=True, 
                                    num_threads=2) # K - кол-во билжайших соседей

    model.fit(csr_matrix(data).T.tocsr(),  # На вход item-user matrix
              show_progress=True)
    
    return model

In [20]:
factors = [100, 150, 200]
regularizations = [0.001, 0.01, 0.1]
iterations = [10, 20, 30]

In [21]:
summary = []

In [22]:
%%time

for factor, regularization, iteration in itertools.product(factors, regularizations, iterations):
            
    print("factors:", factor, "| regularization:", regularization, "| iterations:", iteration)
            
    start_time = time()
    model = create_model(user_item_matrix_bm25, factor, regularization, iteration)
            
    finish_time = time() - start_time
            
    result_test['test'] = result_test['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
    result_train['train'] = result_train['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
            
    test_precision = result_test.apply(lambda row: precision_at_k(row['test'], row['actual']), axis=1).mean()
    train_precision = result_train.apply(lambda row: precision_at_k(row['train'], row['actual']), axis=1).mean()
            
    summary.append([factor, regularization, iteration, finish_time, test_precision, train_precision])

factors: 100 | regularization: 0.001 | iterations: 10


  0%|          | 0/10 [00:00<?, ?it/s]

factors: 100 | regularization: 0.001 | iterations: 20


  0%|          | 0/20 [00:00<?, ?it/s]

factors: 100 | regularization: 0.001 | iterations: 30


...

factors: 200 | regularization: 0.1 | iterations: 20


  0%|          | 0/20 [00:00<?, ?it/s]

factors: 200 | regularization: 0.1 | iterations: 30


  0%|          | 0/30 [00:00<?, ?it/s]

Wall time: 1h 26min 11s


In [23]:
summary_df = pd.DataFrame(data=summary, 
                            columns=['factors', 'regularization', 'iterations', 'time', 'test_precision', 'train_precision'])

In [24]:
summary_df.sort_values('test_precision', ascending=False).head(5)

,factors,regularizations,iterations,time,test_precision,train_precision
21,200,0.01,10,7.382517,0.202351,0.912605
24,200,0.10,10,6.927773,0.201567,0.915566
12,150,0.01,10,6.130445,0.201567,0.888996
26,200,0.10,30,20.933895,0.200392,0.919408
25,200,0.10,20,13.673546,0.199804,0.919968
